# Library

In [13]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as ds
import torchvision.transforms as transforms

import lightning as L
from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Pytorch Practice

## Diamonds

In [48]:
# 사용 가능한 GPU가 있는 경우 'cuda'를, 그렇지 않으면 'cpu'를 사용하도록 장치를 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드를 설정하여 실험의 재현성을 보장합니다.
seed = 0
random.seed(seed)
np.random.seed(seed)  # NumPy의 시드를 설정합니다.
torch.manual_seed(seed)  # PyTorch의 시드를 설정합니다.

# 장치가 CUDA(GPU)일 경우, CUDA의 시드를 설정합니다.
if device == 'cuda':
    torch.cuda.manual_seed(seed)  # 현재 장치의 CUDA 시드를 설정합니다.
    torch.cuda.manual_seed_all(seed)  # 모든 CUDA 장치의 시드를 설정합니다.
    torch.backends.cudnn.deterministic = False  # Deterministic 연산을 비활성화합니다. 비활성화하면 더 빠른 연산이 가능할 수 있습니다.
    torch.backends.cudnn.benchmark = True  # 벤치마크를 활성화하여 최적의 성능을 위해 CUDA 커널을 자동으로 튜닝합니다.

# 하이퍼파라미터를 정의합니다.
batch_size = 32  # 데이터 배치의 크기를 설정합니다.
epochs = 100  # 학습 에포크 수를 설정합니다.
learning_rate = 1e-2  # 학습률을 설정합니다.
hidden_dim = 32  # 숨겨진 층의 차원 크기를 설정합니다.

In [14]:
def convert_category_into_integer(df: pd.DataFrame, columns: list):
    """
    주어진 DataFrame의 특정 열들을 범주형에서 정수형으로 변환합니다.
    
    Parameters:
    - df (pd.DataFrame): 변환할 데이터프레임
    - columns (list): 범주형에서 정수형으로 변환할 열 이름의 리스트
    
    Returns:
    - pd.DataFrame: 변환된 데이터프레임
    - dict: 각 열에 대해 적합한 LabelEncoder 객체를 포함하는 딕셔너리
    """
    label_encoders = {}  # 각 열의 LabelEncoder 객체를 저장할 딕셔너리입니다.
    
    for column in columns:
        # 각 열에 대해 LabelEncoder 객체를 생성합니다.
        label_encoder = LabelEncoder()
        
        # LabelEncoder를 사용하여 해당 열의 범주형 데이터를 정수형으로 변환합니다.
        df.loc[:, column] = label_encoder.fit_transform(df[column])
        
        # 변환된 LabelEncoder 객체를 딕셔너리에 저장합니다.
        label_encoders.update({column: label_encoder})
    
    # 변환된 데이터프레임과 LabelEncoder 객체를 포함하는 딕셔너리를 반환합니다.
    return df, label_encoders

In [255]:
# Seaborn에서 'diamonds' 데이터셋을 로드합니다.
diamonds = sns.load_dataset('diamonds')

# 범주형 변수를 정수형으로 변환하는 함수 `convert_category_into_integer`를 호출합니다.
# 'cut', 'color', 'clarity' 열의 범주형 데이터를 정수형으로 변환합니다.
diamonds, _ = convert_category_into_integer(diamonds, ('cut', 'color', 'clarity'))

# 결측치 수를 각 열별로 출력합니다.
print(diamonds.isna().sum())

# 중복된 행의 수를 출력합니다.
print(diamonds.duplicated().sum())

# 중복된 행을 제거하고 인덱스를 재설정합니다.
diamonds = diamonds.drop_duplicates().reset_index(drop=True)

# 데이터를 훈련 세트(60%)와 나머지 세트(40%)로 분리합니다.
train, temp = train_test_split(diamonds, test_size=0.4, random_state=seed)

# 나머지 세트를 검증 세트(50%)와 테스트 세트(50%)로 분리합니다.
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

# 훈련, 검증, 테스트 세트의 샘플 수를 출력합니다.
print(len(train), len(valid), len(test))

# 특성 열을 표준화하기 위한 StandardScaler를 초기화합니다.
standard_scaler = StandardScaler()

# 훈련 세트의 'carat', 'depth', 'table', 'price', 'x', 'y', 'z' 열을 표준화합니다.
# 표준화는 훈련 데이터의 평균과 표준편차를 사용하여 수행됩니다.
train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.fit_transform(train.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 검증 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(valid.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

# 테스트 세트의 동일한 열을 훈련 세트에서 계산된 평균과 표준편차를 사용하여 표준화합니다.
test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] = \
    standard_scaler.transform(test.loc[:, ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']] )

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64
146
32276 10759 10759


C:\Users\USER\AppData\Local\Temp\ipykernel_7312\1092303787.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 3 1 ... 4 3 2]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.

C:\Users\USER\AppData\Local\Temp\ipykernel_7312\1092303787.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 4 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.

C:\Users\USER\AppData\Local\Temp\ipykernel_7312\1092303787.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3 2 4 ... 2 3 3]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.

C:\Users\USER\AppData\Local\Temp\ipykernel_7312\1341279636.py:31: FutureWarning:

Setting an item of incompatible d

In [256]:
class DiamondsDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'cut' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.iloc[idx].drop('cut').values).float()
        
        # 'cut' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.Tensor([self.data.iloc[idx].cut]).long()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

In [257]:
# 학습 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = DiamondsDataset(train)

# 검증 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = DiamondsDataset(valid)

# 테스트 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
test_dataset = DiamondsDataset(test)

In [86]:
# 학습 데이터셋을 위한 DataLoader를 설정합니다.
train_dataloader = DataLoader(
    dataset=train_dataset,  # 학습 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=True,  # 데이터셋을 매 에포크마다 무작위로 섞습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 검증 데이터셋을 위한 DataLoader를 설정합니다.
valid_dataloader = DataLoader(
    dataset=valid_dataset,  # 검증 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 검증 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 테스트 데이터셋을 위한 DataLoader를 설정합니다.
test_dataloader = DataLoader(
    dataset=test_dataset,  # 테스트 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 테스트 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

In [258]:
class Model(nn.Module):  # nn.Module을 상속받아 새로운 모델 클래스를 정의합니다.
    def __init__(self, input_dim, hidden_dim, output_dim):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.input_dim = input_dim  # 입력 차원 크기를 저장합니다.
        self.hidden_dim = hidden_dim  # 숨겨진 층의 차원 크기를 저장합니다.
        self.output_dim = output_dim  # 출력 차원 크기를 저장합니다.

        self.linear = nn.Linear(input_dim, hidden_dim)  # 입력 차원에서 숨겨진 차원으로의 선형 변환을 정의합니다.
        self.relu = nn.ReLU()  # ReLU 활성화 함수를 정의합니다.
        self.output = nn.Linear(hidden_dim, output_dim)  # 숨겨진 차원에서 출력 차원으로의 선형 변환을 정의합니다.
    
    def forward(self, x):  # 순전파 메서드
        x = self.linear(x)  # 입력 데이터에 대해 선형 변환을 적용합니다.
        x = self.relu(x)  # ReLU 활성화 함수를 적용하여 비선형성을 추가합니다.
        x = self.output(x)  # 두 번째 선형 변환을 적용하여 최종 출력을 계산합니다.

        return x  # 최종 출력을 반환합니다.

In [90]:
# Model 클래스를 사용하여 모델 인스턴스를 생성합니다.
model = Model(
    input_dim=len(diamonds.columns)-1,  # 입력 차원 크기를 설정합니다. diamonds 데이터프레임의 열 개수에서 1을 뺀 값입니다.
    hidden_dim=hidden_dim,  # 숨겨진 층의 차원 크기를 설정합니다. 이 값은 미리 정의된 `hidden_dim` 변수로부터 가져옵니다.
    output_dim=train['cut'].nunique(),  # 출력 차원 크기를 설정합니다. 'cut' 열의 고유한 값의 개수를 가져옵니다.
).to(device)

In [58]:
# model test
model(list(train_dataloader)[0].get('X'))

tensor([[ 1.4337e-01, -3.9626e-01, -1.7633e-01, -6.4472e-02, -3.0898e-02],
        [ 8.1896e-01, -5.6792e-01,  3.3963e-02, -2.8330e-01,  3.1737e-01],
        [ 6.2383e-02, -5.5080e-01, -2.5680e-01,  1.9654e-01,  1.5629e-01],
        [ 1.5736e-01, -8.0738e-01, -2.7905e-01,  2.4286e-01,  2.0203e-01],
        [ 3.5355e-01, -7.3082e-01, -2.9775e-01, -1.8693e-01,  1.6302e-01],
        [ 4.3908e-01, -2.6137e-01, -8.6308e-02, -1.6603e-02, -1.1316e-01],
        [ 4.3533e-01, -5.7853e-01, -2.6183e-01,  2.5193e-01, -2.4184e-02],
        [ 7.3421e-01, -1.5000e-01,  1.4241e-01, -3.4735e-01,  1.9237e-01],
        [ 9.9654e-02, -3.7390e-01, -1.8896e-01,  2.1343e-02, -5.6211e-04],
        [ 1.2355e-01, -5.3569e-01, -3.0869e-01, -1.8574e-01,  2.5175e-01],
        [ 2.0194e-01, -9.5226e-02,  1.2871e-01,  2.7441e-01,  3.2538e-02],
        [ 1.4494e-01, -4.1288e-01, -2.3767e-01, -4.3216e-02,  5.0075e-02],
        [ 4.2626e-01, -9.8128e-02,  6.4640e-02, -2.7806e-01, -1.2381e-01],
        [ 3.6577e-01, -4.

In [61]:
# 손실 함수를 CrossEntropyLoss로 설정합니다.
criterion = nn.CrossEntropyLoss().to(device)
# CrossEntropyLoss는 다중 클래스 분류 문제에서 일반적으로 사용됩니다.
# `device`는 모델과 데이터가 위치할 장치 (예: CPU 또는 GPU)를 지정합니다.

# Adam 최적화 알고리즘을 설정합니다.
optimizer = optim.Adam(
    model.parameters(),  # 최적화할 모델의 파라미터를 지정합니다.
    lr=learning_rate,  # 학습률을 설정합니다. 이 값은 미리 정의된 `learning_rate` 변수로부터 가져옵니다.
)

In [ ]:
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []
for epoch in range(1, epochs+1):
    total_train_loss = 0
    total_train_acc = 0
    for batch in train_dataloader:
        X = batch.get('X').to(device)
        y = batch.get('y').to(device)
        y = y.squeeze()

        optimizer.zero_grad()
        output = model(X)
        logit = F.softmax(output, dim=-1)
        train_loss = criterion(logit, y)
        train_loss.backward()
        optimizer.step()

        predicted_label = logit.argmax(dim=-1)
        acc = (predicted_label==y).float().mean()

        total_train_loss += train_loss
        total_train_acc += acc

    mean_train_loss = total_train_loss / len(train_dataloader)
    mean_train_acc = total_train_acc / len(train_dataloader)
    train_losses.append(mean_train_loss)
    train_accs.append(mean_train_acc)


###################### valid ######################
    total_valid_loss = 0
    total_valid_acc = 0
    for X, y in valid_dataloader:
        X = batch.get('X').to(device)
        y = batch.get('y').to(device)
        y = y.squeeze()

        output = model(X)
        logit = F.softmax(output, dim=-1)
        valid_loss = criterion(logit, y)

        predicted_label = logit.argmax(dim=-1)
        acc = (predicted_label==y).float().mean()

        total_valid_loss += valid_loss
        total_valid_acc += acc

    mean_valid_loss = total_valid_loss / len(valid_dataloader)
    mean_valid_acc = total_valid_acc / len(valid_dataloader)
    valid_losses.append(mean_valid_loss)
    valid_accs.append(mean_valid_acc)

    print(f'Epoch: {epoch} | train_loss: {mean_train_loss: .4f} | train_acc: {mean_train_acc*100: .2f}% | valid_loss: {mean_valid_loss: .4f} | valid_acc: {mean_valid_acc*100: .2f}%')

Regression

In [ ]:
# target: cut -> price
# Loss: CrossEntropy -> MSELoss
# model target: 5 -> 1
# acc -> X

In [259]:
class DiamondsDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'price' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.iloc[idx].drop('price').values).float()
        
        # 'price' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.Tensor([self.data.iloc[idx].price]).float()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

In [260]:
# 학습 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = DiamondsDataset(train)

# 검증 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = DiamondsDataset(valid)

# 테스트 데이터를 DiamondsDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
test_dataset = DiamondsDataset(test)

In [261]:
# 학습 데이터셋을 위한 DataLoader를 설정합니다.
train_dataloader = DataLoader(
    dataset=train_dataset,  # 학습 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=True,  # 데이터셋을 매 에포크마다 무작위로 섞습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 검증 데이터셋을 위한 DataLoader를 설정합니다.
valid_dataloader = DataLoader(
    dataset=valid_dataset,  # 검증 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 검증 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 테스트 데이터셋을 위한 DataLoader를 설정합니다.
test_dataloader = DataLoader(
    dataset=test_dataset,  # 테스트 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 테스트 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

In [262]:
class Model(nn.Module):  # nn.Module을 상속받아 새로운 모델 클래스를 정의합니다.
    def __init__(self, input_dim, hidden_dim, output_dim):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.input_dim = input_dim  # 입력 차원 크기를 저장합니다.
        self.hidden_dim = hidden_dim  # 숨겨진 층의 차원 크기를 저장합니다.
        self.output_dim = output_dim  # 출력 차원 크기를 저장합니다.

        self.linear = nn.Linear(input_dim, hidden_dim)  # 입력 차원에서 숨겨진 차원으로의 선형 변환을 정의합니다.
        self.relu = nn.ReLU()  # ReLU 활성화 함수를 정의합니다.
        self.output = nn.Linear(hidden_dim, output_dim)  # 숨겨진 차원에서 출력 차원으로의 선형 변환을 정의합니다.
    
    def forward(self, x):  # 순전파 메서드
        x = self.linear(x)  # 입력 데이터에 대해 선형 변환을 적용합니다.
        x = self.relu(x)  # ReLU 활성화 함수를 적용하여 비선형성을 추가합니다.
        x = self.output(x)  # 두 번째 선형 변환을 적용하여 최종 출력을 계산합니다.

        return x  # 최종 출력을 반환합니다.

In [263]:
# Model 클래스를 사용하여 모델 인스턴스를 생성합니다.
model = Model(
    input_dim=len(diamonds.columns)-1,  # 입력 차원 크기를 설정합니다. diamonds 데이터프레임의 열 개수에서 1을 뺀 값입니다.
    hidden_dim=hidden_dim,  # 숨겨진 층의 차원 크기를 설정합니다. 이 값은 미리 정의된 `hidden_dim` 변수로부터 가져옵니다.
    output_dim=1,  # 출력 차원 크기를 설정합니다. 'cut' 열의 고유한 값의 개수를 가져옵니다.
).to(device)

In [264]:
# 평균 제곱 오차 (MSE) 손실 함수를 정의하고, 이를 지정된 장치 (예: GPU 또는 CPU)로 이동
criterion = nn.MSELoss().to(device)

# Adam 최적화 알고리즘을 설정합니다.
optimizer = optim.Adam(
    model.parameters(),  # 최적화할 모델의 파라미터를 지정합니다.
    lr=learning_rate,  # 학습률을 설정합니다. 이 값은 미리 정의된 `learning_rate` 변수로부터 가져옵니다.
)

In [265]:
# 학습 손실과 정확도, 검증 손실과 정확도를 저장할 리스트를 초기화
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

# 에포크 수만큼 루프를 실행
for epoch in range(1, epochs+1):
    # 학습 손실과 정확도를 초기화
    total_train_loss = 0
    total_train_acc = 0

    # 학습 데이터로 배치 단위로 루프를 실행
    for batch in train_dataloader:
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동

        optimizer.zero_grad()  # 기울기 초기화
        output = model(X)  # 모델의 예측값을 계산
        train_loss = criterion(output, y)  # 손실 계산
        train_loss.backward()  # 기울기 계산
        optimizer.step()  # 파라미터 업데이트

        total_train_loss += train_loss.item()  # 손실 누적

    # 평균 학습 손실과 정확도 계산
    mean_train_loss = total_train_loss / len(train_dataloader)
    train_losses.append(mean_train_loss)  # 학습 손실 기록

    ###################### 검증 ######################
    # 검증 손실과 정확도를 초기화
    total_valid_loss = 0
    total_valid_acc = 0

    # 검증 데이터로 배치 단위로 루프를 실행
    for batch in valid_dataloader:
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동

        output = model(X)  # 모델의 예측값을 계산
        valid_loss = criterion(output, y)  # 검증 손실 계산

        total_valid_loss += valid_loss.item()  # 손실 누적

    # 평균 검증 손실과 정확도 계산
    mean_valid_loss = total_valid_loss / len(valid_dataloader)
    mean_valid_acc = total_valid_acc / len(valid_dataloader)
    valid_losses.append(mean_valid_loss)  # 검증 손실 기록
    valid_accs.append(mean_valid_acc)  # 검증 정확도 기록

    # 에포크마다 학습 손실과 검증 손실 출력
    print(f'Epoch: {epoch} | train_loss: {mean_train_loss:.4f} | valid_loss: {mean_valid_loss:.4f}')

Epoch: 1 | train_loss: 0.2134 | valid_loss: 0.0988
Epoch: 2 | train_loss: 0.0743 | valid_loss: 0.0843
Epoch: 3 | train_loss: 0.0656 | valid_loss: 0.0779
Epoch: 4 | train_loss: 0.0615 | valid_loss: 0.0728
Epoch: 5 | train_loss: 0.0588 | valid_loss: 0.0720
Epoch: 6 | train_loss: 0.0564 | valid_loss: 0.0686
Epoch: 7 | train_loss: 0.0544 | valid_loss: 0.0688
Epoch: 8 | train_loss: 0.0530 | valid_loss: 0.0712


KeyboardInterrupt: 

In [283]:
# 모델 검증
# 모델 예측 가격을 원본 스케일로 변환 후 실제 값과 예측 값을 시각화 후 분석

results = []
with torch.no_grad():
    for batch in test_dataloader:
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동

        output = model(X)  # 모델의 예측값을 계산
        results.append(output) # 결과 리스트에 현재 배치의 모델 출력을 추가

In [285]:
# results 리스트에 저장된 텐서들을 하나로 합치고, 데이터프레임을 생성
results = pd.DataFrame({
    'y_pred': torch.concat(results)  # 결과 리스트에 저장된 텐서들을 하나로 합침
                .reshape(-1)  # 1차원으로 변환
                .numpy(),  # 넘파이 배열로 변환
    'ground_truth': test.price.values[:-7]  # 실제 값 (마지막 7개 값은 제외)
})
results

In [294]:
# 표준화된 데이터를 원래 스케일로 변환
# 표준화 스케일을 이용하여 원래의 값으로 되돌림
results = (results * standard_scaler.scale_[3] + standard_scaler.mean_[3]).reset_index()

In [296]:
# 데이터프레임 'results'의 예측값과 실제 값을 산점도로 시각화
px.scatter(
    results,  # 시각화할 데이터프레임
    x='index',  # x축에는 데이터프레임의 인덱스를 사용
    y=['y_pred', 'ground_truth']  # y축에는 예측값과 실제 값을 사용하여 두 개의 시리즈를 표시
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=y_pred<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'y_pred',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'y_pred',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([    0,     1,     2, ..., 10749, 10750, 10751]),
              'xaxis': 'x',
              'y': array([ 1064.82275391,  6312.88867188, 13053.02832031, ...,   857.83935547,
                          12642.33105469,  1974.60974121]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=ground_truth<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'ground_truth',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'ground_truth',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([    0,     1,     2, ..., 10749, 10750, 10751]),
              'xaxis': 'x',
              'y': array([ 1007.,  4671., 11820., ...,   805., 12221.,  1715.]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

## Titanic

In [342]:
# 'titanic' 데이터셋을 로드
titanic = sns.load_dataset('titanic')

# 데이터프레임의 각 열에서 결측값의 수를 계산하여 출력
titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [343]:
# 'deck' 열을 문자열형으로 변환
titanic.deck = titanic.deck.astype(str)

# 데이터프레임의 정보 출력
# 데이터프레임의 구조와 각 열의 데이터 타입을 확인
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         891 non-null    object  
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(6)
memory usage: 86.4+ KB


In [344]:
# 데이터프레임에서 결측값이 있는 모든 행을 제거
titanic = titanic.dropna()

In [345]:
# 데이터프레임의 범주형 열을 정수형으로 변환
# 'sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive' 열을 정수형으로 변환
titanic, _ = convert_category_into_integer(titanic, ('sex', 'embarked', 'class', 'who', 'deck', 'embark_town', 'alive'))

C:\Users\USER\AppData\Local\Temp\ipykernel_7312\1092303787.py:20: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 2 0 2 0 2 2 1 2 0 2 2 2 1 2 2 1 1 2 0 2 2 0 0 1 0 0 2 2 2 2 1 1 2 2 2
 2 0 1 0 1 2 1 2 2 0 2 1 2 2 2 1 2 1 2 2 2 1 2 2 2 0 1 2 2 0 2 2 2 0 2 2 0
 0 1 1 2 0 2 2 2 2 2 0 2 2 2 2 2 2 1 0 2 1 1 1 0 2 2 2 2 2 2 1 1 1 0 0 2 0
 2 2 2 1 1 2 2 1 1 1 0 2 2 0 2 2 2 1 2 2 2 2 2 2 0 2 2 2 0 2 0 1 2 2 1 2 0
 2 2 1 1 2 1 0 0 2 1 2 2 2 2 2 2 2 2 0 2 1 2 1 0 2 1 0 1 2 1 2 0 2 1 2 1 0
 2 1 2 1 1 1 1 1 1 2 2 0 2 1 0 1 2 0 2 2 2 0 0 1 2 0 0 1 2 2 0 0 2 1 0 0 2
 2 2 2 2 2 2 2 2 2 1 2 0 0 1 2 2 2 0 0 2 0 0 1 0 0 0 1 2 1 2 1 1 0 0 2 2 1
 1 0 2 1 2 0 0 0 2 0 0 2 0 1 0 1 1 1 1 1 2 2 2 2 2 2 0 1 2 1 2 2 2 0 0 0 2
 2 0 2 2 0 2 2 0 2 2 0 1 2 1 1 0 2 2 0 2 2 2 1 1 1 2 2 2 2 2 1 2 1 2 0 2 1
 1 1 2 2 2 2 2 1 1 2 0 1 2 0 0 2 1 0 1 1 2 2 1 0 1 0 2 0 1 0 0 2 0 1 0 2 0
 1 2 0 2 2 1 1 2 1 2 2 2 2 2 2 0 0 0 2 2 2 0 0 2 0 0 2 2 2 2 0 0 

In [346]:
# 데이터프레임의 정보 출력
# 데이터프레임의 구조와 각 열의 데이터 타입을 확인
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     712 non-null    int64  
 1   pclass       712 non-null    int64  
 2   sex          712 non-null    object 
 3   age          712 non-null    float64
 4   sibsp        712 non-null    int64  
 5   parch        712 non-null    int64  
 6   fare         712 non-null    float64
 7   embarked     712 non-null    object 
 8   class        712 non-null    int64  
 9   who          712 non-null    object 
 10  adult_male   712 non-null    bool   
 11  deck         712 non-null    object 
 12  embark_town  712 non-null    object 
 13  alive        712 non-null    object 
 14  alone        712 non-null    bool   
dtypes: bool(2), float64(2), int64(5), object(6)
memory usage: 79.3+ KB


In [347]:
# 데이터프레임의 모든 열을 float32 데이터 타입으로 변환
titanic = titanic.astype(np.float32)

# 데이터프레임의 정보 출력
# 데이터프레임의 구조와 각 열의 데이터 타입을 확인
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     712 non-null    float32
 1   pclass       712 non-null    float32
 2   sex          712 non-null    float32
 3   age          712 non-null    float32
 4   sibsp        712 non-null    float32
 5   parch        712 non-null    float32
 6   fare         712 non-null    float32
 7   embarked     712 non-null    float32
 8   class        712 non-null    float32
 9   who          712 non-null    float32
 10  adult_male   712 non-null    float32
 11  deck         712 non-null    float32
 12  embark_town  712 non-null    float32
 13  alive        712 non-null    float32
 14  alone        712 non-null    float32
dtypes: float32(15)
memory usage: 47.3 KB


In [348]:
# 원본 데이터셋을 학습용 데이터와 임시 데이터로 분할
# 전체 데이터의 40%를 임시 데이터로, 60%를 학습용 데이터로 사용
train, temp = train_test_split(titanic, test_size=0.4, random_state=seed)

# 임시 데이터를 검증용 데이터와 테스트용 데이터로 분할
# 임시 데이터의 절반을 검증용 데이터로, 나머지 절반을 테스트용 데이터로 사용
valid, test = train_test_split(temp, test_size=0.5, random_state=seed)

In [349]:

class TitanicDataset(Dataset):
    def __init__(self, data):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.data = data  # 데이터프레임을 저장합니다.
    
    def __len__(self):
        return len(self.data)  # 데이터셋의 전체 샘플 수를 반환합니다.
    
    def __getitem__(self, idx):
        # 인덱스 `idx`에 해당하는 데이터 샘플을 반환합니다.
        
        # 데이터프레임에서 'survived' 열을 제외한 특성값을 가져와서 NumPy 배열로 변환한 뒤, PyTorch 텐서로 변환합니다.
        X = torch.from_numpy(self.data.drop('survived', axis=1).iloc[idx].values).float()
        
        # 'survived' 열의 값을 텐서로 변환하여 레이블을 생성합니다.
        y = torch.tensor(self.data.iloc[idx].survived).long()
        
        # 입력 데이터와 레이블을 딕셔너리 형태로 반환합니다.
        return {
            'X': X,
            'y': y,
        }

In [350]:
# 학습 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
train_dataset = TitanicDataset(train)

# 검증 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
valid_dataset = TitanicDataset(valid)

# 테스트 데이터를 TitanicDataset 클래스를 사용하여 데이터셋 객체로 변환합니다.
test_dataset = TitanicDataset(test)

In [351]:
# 학습 데이터셋을 위한 DataLoader를 설정합니다.
train_dataloader = DataLoader(
    dataset=train_dataset,  # 학습 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=True,  # 데이터셋을 매 에포크마다 무작위로 섞습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 검증 데이터셋을 위한 DataLoader를 설정합니다.
valid_dataloader = DataLoader(
    dataset=valid_dataset,  # 검증 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 검증 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

# 테스트 데이터셋을 위한 DataLoader를 설정합니다.
test_dataloader = DataLoader(
    dataset=test_dataset,  # 테스트 데이터셋을 지정합니다.
    batch_size=batch_size,  # 데이터 배치의 크기를 설정합니다.
    shuffle=False,  # 테스트 데이터는 무작위로 섞지 않습니다.
    drop_last=True,  # 데이터셋의 크기가 배치 크기로 나누어떨어지지 않을 때 마지막 배치를 버립니다.
)

In [352]:
class Model(nn.Module):  # nn.Module을 상속받아 새로운 모델 클래스를 정의합니다.
    def __init__(self, input_dim, hidden_dim, output_dim):  # 생성자 메서드
        super().__init__()  # 부모 클래스의 생성자를 호출하여 초기화합니다.
        self.input_dim = input_dim  # 입력 차원 크기를 저장합니다.
        self.hidden_dim = hidden_dim  # 숨겨진 층의 차원 크기를 저장합니다.
        self.output_dim = output_dim  # 출력 차원 크기를 저장합니다.

        self.linear = nn.Linear(input_dim, hidden_dim)  # 입력 차원에서 숨겨진 차원으로의 선형 변환을 정의합니다.
        self.relu = nn.ReLU()  # ReLU 활성화 함수를 정의합니다.
        self.output = nn.Linear(hidden_dim, output_dim)  # 숨겨진 차원에서 출력 차원으로의 선형 변환을 정의합니다.
    
    def forward(self, x):  # 순전파 메서드
        x = self.linear(x)  # 입력 데이터에 대해 선형 변환을 적용합니다.
        x = self.relu(x)  # ReLU 활성화 함수를 적용하여 비선형성을 추가합니다.
        x = self.output(x)  # 두 번째 선형 변환을 적용하여 최종 출력을 계산합니다.

        return x  # 최종 출력을 반환합니다.

In [368]:
# len(titanic.columns) - 1: 입력 특성의 수 (타이타닉 데이터셋에서 'survived' 열을 제외한 나머지 열)
# hidden_dim: 은닉층의 뉴런 수 (변수로 설정된 값)
# 2: 출력 클래스의 수 (이진 분류 문제이므로 두 개 클래스)
model = Model(len(titanic.columns) - 1, hidden_dim, 2)

In [369]:
# 교차 엔트로피 손실 함수 (CrossEntropyLoss)를 정의
# 이 손실 함수는 주로 다중 클래스 분류 문제에서 사용용
# 손실 함수를 정의한 후, 이를 지정된 장치 (예: GPU 또는 CPU)로 이동
criterion = nn.CrossEntropyLoss().to(device)

# Adam 최적화 알고리즘을 설정합니다.
optimizer = optim.Adam(
    model.parameters(),  # 최적화할 모델의 파라미터를 지정합니다.
    lr=learning_rate,  # 학습률을 설정합니다. 이 값은 미리 정의된 `learning_rate` 변수로부터 가져옵니다.
)

In [370]:
# 학습 손실과 정확도, 검증 손실과 정확도를 저장할 리스트를 초기화
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

# 에포크 수만큼 루프를 실행
for epoch in range(1, epochs + 1):
    # 학습 손실과 정확도를 초기화
    total_train_loss = 0
    total_train_acc = 0

    # 학습 데이터로 배치 단위로 루프를 실행
    model.train()  # 모델을 학습 모드로 설정
    for batch in train_dataloader:
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        optimizer.zero_grad()  # 기울기 초기화
        output = model(X)  # 모델의 예측값을 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수를 적용
        train_loss = criterion(logit, y)  # 손실 계산
        train_loss.backward()  # 손실에 대한 기울기 계산
        optimizer.step()  # 모델 파라미터 업데이트

        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
        acc = (predicted_label == y).float().mean()  # 정확도 계산

        total_train_loss += train_loss.item()  # 손실 합산
        total_train_acc += acc.item()  # 정확도 합산

    # 에포크 당 평균 학습 손실과 정확도 계산
    mean_train_loss = total_train_loss / len(train_dataloader)
    mean_train_acc = total_train_acc / len(train_dataloader)
    train_losses.append(mean_train_loss)  # 학습 손실 리스트에 추가
    train_accs.append(mean_train_acc)  # 학습 정확도 리스트에 추가

    # 검증 데이터로 배치 단위로 루프를 실행
    total_valid_loss = 0
    total_valid_acc = 0
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():  # 기울기 계산을 하지 않도록 설정
        for batch in valid_dataloader:
            X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
            y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
            y = y.squeeze()  # 레이블의 차원을 축소

            output = model(X)  # 모델의 예측값을 계산
            logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수를 적용
            valid_loss = criterion(logit, y)  # 검증 손실 계산

            predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
            acc = (predicted_label == y).float().mean()  # 정확도 계산

            total_valid_loss += valid_loss.item()  # 검증 손실 합산
            total_valid_acc += acc.item()  # 검증 정확도 합산

        # 에포크 당 평균 검증 손실과 정확도 계산
        mean_valid_loss = total_valid_loss / len(valid_dataloader)
        mean_valid_acc = total_valid_acc / len(valid_dataloader)
        valid_losses.append(mean_valid_loss)  # 검증 손실 리스트에 추가
        valid_accs.append(mean_valid_acc)  # 검증 정확도 리스트에 추가

    # 에포크별 손실과 정확도 출력
    print(f'Epoch: {epoch} | train_loss: {mean_train_loss:.4f} | train_acc: {mean_train_acc * 100:.2f}% | valid_loss: {mean_valid_loss:.4f} | valid_acc: {mean_valid_acc * 100:.2f}%')

Epoch: 1 | train_loss:  0.6550 | train_acc:  65.10% | valid_loss:  0.6530 | valid_acc:  63.28%
Epoch: 2 | train_loss:  0.6067 | train_acc:  69.79% | valid_loss:  0.6473 | valid_acc:  63.28%
Epoch: 3 | train_loss:  0.6047 | train_acc:  68.49% | valid_loss:  0.6461 | valid_acc:  64.84%
Epoch: 4 | train_loss:  0.6051 | train_acc:  67.97% | valid_loss:  0.6459 | valid_acc:  64.84%
Epoch: 5 | train_loss:  0.6077 | train_acc:  68.75% | valid_loss:  0.6467 | valid_acc:  64.06%
Epoch: 6 | train_loss:  0.5960 | train_acc:  69.79% | valid_loss:  0.6470 | valid_acc:  64.06%
Epoch: 7 | train_loss:  0.5917 | train_acc:  70.83% | valid_loss:  0.6459 | valid_acc:  66.41%
Epoch: 8 | train_loss:  0.6003 | train_acc:  69.53% | valid_loss:  0.6444 | valid_acc:  65.62%
Epoch: 9 | train_loss:  0.5906 | train_acc:  71.61% | valid_loss:  0.6431 | valid_acc:  64.84%
Epoch: 10 | train_loss:  0.6025 | train_acc:  70.57% | valid_loss:  0.6420 | valid_acc:  65.62%
Epoch: 11 | train_loss:  0.5971 | train_acc:  70.

# Pytorch Lightning

PyTorch Lightning은 PyTorch의 강력함을 유지하면서도 코드의 재사용성과 관리 편의성을 크게 개선시킨 <br>
주로 복잡한 훈련 루프를 추상화하여 코드의 간결성을 높이고, 디버깅과 스케일링을 용이하게 만드는 것을 주된 목표로 함 <br>

<br>

<span style="font-size: 20pt;"> 장점 </span> 

1. 모듈화된 구조: PyTorch Lightning은 훈련, 검증, 테스트 루프와 옵티마이저, 데이터 로더 등을 각각의 모듈로 분리하여 제공 <br>
&nbsp;&nbsp;&nbsp;&nbsp; -> 훨씬 읽기 쉽고 관리하기 쉽게 만들어 줌 (코드의 간결성 확보)
2. 자동화된 훈련 루프: LightningModule 클래스를 상속받아 모델을 정의할 때, 훈련 루프에서 필요한 많은 부분들(forward, loss, optimizer 등)이 자동으로 처리 <br>
&nbsp;&nbsp;&nbsp;&nbsp; -> 사용자가 코드의 일부를 더 간결하게 작성할 수 있도록 해줌
3. 스케일링과 분산 훈련 지원: multi-GPU나 분산 훈련을 지원하여 더 큰 데이터셋이나 복잡한 모델을 쉽게 처리할 수 있게 해줌
5. 확장성: 기존의 PyTorch 코드를 LightningModule로 변환하는 과정이 비교적 간단하고, 다양한 연구와 프로젝트에 쉽게 적용할 수 있음

## 설치 방법

In [ ]:
!pip install lightning

## 사용 방법

### Hyperparameters

In [385]:
# 배치 크기 (Batch Size): 한 번의 업데이트에서 사용할 데이터 샘플 수
batch_size = 128

# 에포크 수 (Epochs): 전체 데이터셋을 몇 번 반복하여 학습할 것인지 지정
epochs = 20

# 학습률 (Learning Rate): 모델 파라미터를 업데이트할 때 사용되는 학습률
learning_rate = 1e-3

# 은닉층의 뉴런 수 (Hidden Dimension): 은닉층의 차원 또는 뉴런 수
hidden_dim = 64

### LightningDataModule

학습 시 사용할 DataLoader를 호출하는 클래스 <br>
관련된 전처리 등을 내부에서 처리하여 사용 <br>

<br>

<span style="font-size:150%">코드</span>

> ```python
> class CustomDataModule(L.LightningDataModule):
>     def __init__(
>         self,
>         batch_size: int,
>         num_workers: int,
>         ):
>         super().__init__()
>         self.num_workers = 4
>         self.batch_size = batch_size
> 
>     def prepare(self, train, valid, test):     
>         self.train, self.valid, self.test = train, valid, test
> 
>     def setup(self, stage: str):
>         if stage == "fit":      
>             self.train_data = self.train
>             self.valid_data = self.valid
> 
>         if stage == "test":     
>             self.test_data = self.test
> 
>     def train_dataloader(self):
>         return DataLoader(
>             dataset=self.train_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )
> 
>     def val_dataloader(self):
>         return DataLoader(
>             dataset=self.val_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )
> 
>     def test_dataloader(self):
>         return DataLoader(
>             dataset=self.test_data,
>             batch_size=self.batch_size,
>             shuffle=False,
>         )

In [386]:
class TitanicDataModule(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        ):
        super().__init__()
        self.batch_size = batch_size

    def prepare(self, train_dataset, valid_dataset, test_dataset):     
        self.train_dataset, self.valid_dataset, self.test_dataset = train_dataset, valid_dataset, test_dataset

    def setup(self, stage: str):
        if stage == "fit":      
            self.train_data = self.train_dataset
            self.valid_data = self.valid_dataset

        if stage == "test":     
            self.test_data = self.test_dataset

    def train_dataloader(self):
        return DataLoader(
            dataset=self.train_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=self.valid_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
        )

In [387]:
titanic_data_module = TitanicDataModule(batch_size=batch_size)
titanic_data_module.prepare(train_dataset, valid_dataset, test_dataset)

### LightningModule

학습 과정에 사용될 코드를 모듈화 한 클래스 <br>
self.log()를 통해 logging 사용 가능 

<br>

<span style="font-size:150%">코드</span>

> ```python
> class LightningMLP(L.LightningModule):
>     def __init__(
>         self,
>         model: torch.nn,    # 구축한 모델
>         learning_rate: float,
>         ):
>         super().__init__()
>         self.model = model
>         self.learning_rate = learning_rate
>     
>     def training_step(self, batch, batch_idx):      # train 코드
>         pass
>     
>     def on_train_epoch_end(self, *args, **kwargs):  # train이 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
>         pass
>     
>     def validation_step(self, *args, **kwargs):     # validation 코드
>         pass
>     
>     def on_validation_epoch_end(self):              # valid가 1 epoch 끝날 때 실행될 코드, 주로 metric 출력을 위해 사용
>         pass
>     
>     def test_step(self, *args, **kwargs):           # test 코드
>         pass
>     
>     def configure_optimizers(self):                 # optimizer 설정
>         pass
> ```

In [401]:
class TitanicModule(L.LightningModule):
    def __init__(
        self,
        model: torch.nn.Module,    # 구축한 모델
        learning_rate: float,      # 학습률
    ):
        super().__init__()
        self.model = model         # 모델 초기화
        self.learning_rate = learning_rate  # 학습률 초기화

        self.total_train_loss = []  # 학습 손실을 저장할 리스트
        self.total_train_acc = []   # 학습 정확도를 저장할 리스트
        self.total_valid_loss = []  # 검증 손실을 저장할 리스트
        self.total_valid_acc = []   # 검증 정확도를 저장할 리스트
    
    def training_step(self, batch, batch_idx):
        # 학습 단계에서 호출되는 메서드
        if batch_idx == 0:
            self.total_train_loss.clear()  # 손실 리스트 초기화
            self.total_train_acc.clear()   # 정확도 리스트 초기화

        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        loss = F.cross_entropy(logit, y)  # 손실 계산

        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
        acc = (predicted_label == y).float().mean()  # 정확도 계산

        self.total_train_loss.append(loss.detach().cpu().numpy())  # 학습 손실 리스트에 추가
        self.total_train_acc.append(acc.detach().cpu().numpy())    # 학습 정확도 리스트에 추가

        return loss  # 손실 반환
    
    def on_train_epoch_end(self, *args, **kwargs):
        # 학습 에포크가 끝날 때 호출되는 메서드
        print(f'Train Loss: {np.mean(self.total_train_loss): .4f} | Train Acc: {np.mean(self.total_train_acc) * 100: .2f}%')
    
    def validation_step(self, batch, batch_idx):
        # 검증 단계에서 호출되는 메서드
        if batch_idx == 0:
            self.total_valid_loss.clear()  # 검증 손실 리스트 초기화
            self.total_valid_acc.clear()   # 검증 정확도 리스트 초기화

        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        loss = F.cross_entropy(logit, y)  # 검증 손실 계산

        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산
        acc = (predicted_label == y).float().mean()  # 정확도 계산

        self.total_valid_loss.append(loss.detach().cpu().numpy())  # 검증 손실 리스트에 추가
        self.total_valid_acc.append(acc.detach().cpu().numpy())    # 검증 정확도 리스트에 추가

        return loss  # 검증 손실 반환
    
    def on_validation_epoch_end(self):
        # 검증 에포크가 끝날 때 호출되는 메서드
        print(f'Valid Loss: {np.mean(self.total_valid_loss): .4f} | Valid Acc: {np.mean(self.total_valid_acc) * 100: .2f}%')
    
    def test_step(self, batch, batch_idx):
        # 테스트 단계에서 호출되는 메서드
        X = batch.get('X').to(device)  # 입력 데이터를 장치로 이동
        y = batch.get('y').to(device)  # 레이블 데이터를 장치로 이동
        y = y.squeeze()  # 레이블의 차원을 축소

        output = self.model(X)  # 모델의 예측값 계산
        logit = F.softmax(output, dim=-1)  # 소프트맥스 활성화 함수 적용
        predicted_label = logit.argmax(dim=-1)  # 예측된 레이블 계산

        return predicted_label  # 예측된 레이블 반환

    def configure_optimizers(self):
        # 옵티마이저를 설정하는 메서드
        optimizer = optim.Adam(
            self.model.parameters(),  # 모델 파라미터를 옵티마이저에 전달
            lr=self.learning_rate,    # 학습률 설정
        )

        return {'optimizer': optimizer}  # 옵티마이저 반환

In [402]:
# TitanicModule 클래스의 인스턴스를 생성합니다.
titanic_module = TitanicModule(
    model=model,              # 학습할 모델 인스턴스 (예: Model 객체)
    learning_rate=learning_rate  # 학습률 (예: 1e-3)
)

### Trainer

학습에 필요한 내용을 정의하는 클래스

<br>

<span style="font-size:150%">코드</span>

> ```python
> trainer = Trainer(
>     # accelerator='gpu',    # gpu 사용 시 
>     # devices=2,            # gpu 사용 시 장치 위치 입력 ex) 0, 1, 2, [0, 1], [0, 3], ...
>     max_epochs=epochs,
>     callbacks=[             # callback
>         EarlyStopping(monitor='val_loss', mode='min', patience=10)], # val_loss를 기준으로 최솟값이 10번 이상 업데이트되지 않으면 학습 중지
>     log_every_n_steps=1,
> )
> ```

<br>

생성된 trainer 객체를 이용하여 학습 진행

> ```python
> trainer.fit(
>     model=custom_model,
>     datamodule=custom_data_module,
> )
> ```

In [403]:
# Trainer 인스턴스 생성
trainer = Trainer(
    max_epochs=epochs,  # 학습할 최대 에포크 수
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [404]:
# PyTorch Lightning Trainer를 사용하여 모델 학습을 시작
# 'titanic_module'은 학습할 모델과 학습 및 검증 루프를 정의한 LightningModule 인스턴스
# 'titanic_data_module'은 데이터셋을 로드하고 데이터로더를 제공하는 LightningDataModule 인스턴스
trainer.fit(
    model=titanic_module,       # 학습할 모델 인스턴스
    datamodule=titanic_data_module,  # 데이터셋과 데이터로더를 제공하는 데이터 모듈 인스턴스
)


  | Name  | Type  | Params | Mode
---------------------------------------
0 | model | Model | 1.1 K  | eval
---------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)
0         Modules in train mode
4         Modules in eval mode


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 234.63it/s]Valid Loss:  0.6422 | Valid Acc:  60.94%
                                                                            

c:\ProgramData\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

c:\ProgramData\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

c:\ProgramData\anaconda3\envs\deep_learning\lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: PossibleUserWarning:

The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training

Epoch 19: 100%|██████████| 4/4 [00:00<00:00, 25.35it/s, v_num=9]

`Trainer.fit` stopped: `max_epochs=20` reached.


Train Loss:  0.4899 | Train Acc:  86.93%
Epoch 19: 100%|██████████| 4/4 [00:00<00:00, 24.72it/s, v_num=9]


### Test

학습이 진행되면서 checkpoint가 생성되는데, 이를 통해 학습된 모델 사용 가능

<br>

<span style="font-size:150%">코드</span>

> ```python
> model = CustomDataModule.load_from_checkpoint(path)
> ```

<br>
<br>
<br>

학습된 모델을 가지고 테스트 데이터에 대해 아래와 같이 사용 가능

<br>

<span style="font-size:150%">코드</span>

> ```python
> predictions = trainer.predict(model: L.LightningModule, data_loader)
> ```